# Machines Manufacturing Captal Budgeting Model (Project 1)

Insert your description of the model here and add any additional sections below:
- [**Setup**](#Setup): Runs any imports and other setup
- [**Inputs**](#Inputs): Defines the inputs for the model

## Setup

Setup for the later calculations are here. The necessary packages are imported.

In [40]:
from dataclasses import dataclass
import numpy_financial as npf 

## Inputs

All of the inputs for the model are defined here. A class is constructed to manage the data, and an instance of the class containing the default inputs is created.

In [41]:
@dataclass
class ModelInputs:
    n_phones: float = 100000
    price_scrap: float = 50000
    price_phone: float = 2000
    cost_machine_adv: float = 1000000
    cogs_phone: float = 250
    n_life: int = 10
    n_machines: int = 5
    d_1: float = 100000
    g_d: float = 0.2
    max_year: float = 20
    interest: float = 0.05
        
    # Inputs for bonus problem
    elasticity: float = 100
    demand_constant: float = 300000
        
model_data = ModelInputs()
model_data

ModelInputs(n_phones=100000, price_scrap=50000, price_phone=2000, cost_machine_adv=1000000, cogs_phone=250, n_life=10, n_machines=5, d_1=100000, g_d=0.2, max_year=20, interest=0.05, elasticity=100, demand_constant=300000)

## Revenues
Revenues generated by the company each year consist of goods of sales and 
sales of depreciated fixed assets 

In [42]:
# How many machines in operation per year
def op_sp_machines(current_year):
    '''
    return the number of machines in operation each year
    and whether advertisement is launched this year
    '''
    op = []
    sp = []

    if current_year <= model_data.n_machines:
        op.append(range(1, current_year))    # purchase one machine before year 5
    elif 1+model_data.n_life >= current_year > model_data.n_machines:
        op.append(range(1, model_data.n_machines))
    elif model_data.n_machines+model_data.n_life >= current_year > 1+model_data.n_life:
        sp.append(1)   # add depreciated machines to scraped list
    else:
        op = []
    
    return op, sp

In [43]:
def is_ad(current_year):
    is_ad = False

    if current_year > model_data.n_machines:
        is_ad = True

    return is_ad

In [44]:
# Revenues per year
def total_revenues(n_goods_sold, sp_machines):
    '''
    return the total revenue generated by 
    the company per year
    '''

    sales_from_phone = n_goods_sold * model_data.price_phone
    if sp_machines > 0:     # if there is any machine in the scraped list
        sales_from_machine = model_data.price_scrap
    else:
        sales_from_machine = 0

    revenues = sales_from_machine + sales_from_phone

    return revenues

In [45]:
# Determine the number of goods sold
def goods_sold(production, demand):
    '''
    compare which figure is smaller:
    production of the phones of demand of the phones
    '''
    return production if production <= demand else demand

## Costs
Costs consist of phones manufacture, purchasing machines and advertisement

In [46]:
# cost of purchasing machines
def machines_cost(current_year):
    '''
    only purcahse machines before year 6
    '''
    if current_year <= model_data.n_machines:
        c_machines = model_data.cost_machine_adv
    else:
        c_machines = 0
    return c_machines

In [47]:
def total_costs(n_goods_sold, machines_cost, ad_cost):
    phone_cost = n_goods_sold * model_data.cogs_phone
    costs = machines_cost + ad_cost + phone_cost
    return costs

## Calculating cash flows

In [48]:
def cashflows():
    '''
    return a list that contains each year's cashflow
    '''
    cashflows = []
    demand = model_data.d_1
    
    for i in range(1, model_data.max_year+1):

        op_machines = len(op_sp_machines(i)[0])  # number of operating machines 
        sp_machines = len(op_sp_machines(i)[1])   # number of scraped machines

        if is_ad(i) == True:
            demand *= (1 + model_data.g_d)  # demand of phones if there are ads
            c_ad = model_data.cost_machine_adv  # cost of advertisement
        else:
            demand = model_data.d_1     # demand of phones if no ads
            c_ad = 0

        phones_production = op_machines * model_data.n_phones    # how many phones have been produced 
        n_goods_sold = goods_sold(phones_production, demand)
        revenues = total_revenues(n_goods_sold, sp_machines)     # total reveneus per year

        c_machines = machines_cost(i)
        costs = total_costs(n_goods_sold, c_machines, c_ad)

        cashflow = revenues - costs
        cashflows.append(cashflow)
    
    return cashflows


In [49]:
cash_flows = cashflows()
cash_flows

[174000000,
 174000000,
 174000000,
 174000000,
 174000000,
 174000000,
 174000000,
 174000000,
 174000000,
 174000000,
 174000000,
 -950000,
 -950000,
 -950000,
 -950000,
 -1000000,
 -1000000,
 -1000000,
 -1000000,
 -1000000]

In [50]:
npv = npf.npv(model_data.interest, cash_flows)  # this should ultimately be set to the overally model npv number 
npv

1513327137.0482202